In [58]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

Load the data

In [59]:
!mkdir data && wget https://github.com/sdip15fa/weather-predict/raw/master/data/processed_data.csv -O data/processed_data.csv

input_file = 'data/processed_data.csv'
df = pd.read_csv('data/processed_data.csv',
                 parse_dates=['DateTime'], index_col='DateTime')
df = df.resample('60T').mean()
# df["DateTime"] = pd.to_datetime(df.index)
df = df.reset_index()

mkdir: cannot create directory ‘data’: File exists


Convert the 'Date' and 'Time' columns to integers

In [60]:
df["Temperature"] = df["Temperature"].astype(float)
df["Wind Speed"] = df["Wind Speed"].astype(float)
df["Rainfall"] = df["Rainfall"].astype(float)
df["Wind Direction"] = df["Wind Direction"].astype(float)
df["Humidity"] = df["Humidity"].astype(float)

Filter and replace outliers

In [61]:
df['Temperature'] = df['Temperature'].replace(3276.7, np.nan)
df['Wind Speed'] = df['Wind Speed'].replace(32767, np.nan)
df['Rainfall'] = df['Rainfall'].replace(32767, np.nan)
df['Wind Direction'] = df['Wind Direction'].replace(32767, np.nan)
df['Humidity'] = df['Humidity'].replace(32767, np.nan)
df['Temperature'] = df['Temperature'].mask((df['Temperature'] >= df['Temperature'].quantile(0.995)), np.nan)
df['Wind Speed'] = df['Wind Speed'].mask((df['Wind Speed'] >= df['Wind Speed'].quantile(0.995)) | df["Wind Speed"] < 0, np.nan)
df['Rainfall'] = df['Rainfall'].mask((df['Rainfall'] >= df['Rainfall'].quantile(0.995))| df["Rainfall"] < 0, np.nan)
df['Wind Direction'] = df['Wind Direction'].mask((df['Wind Direction'] < 0) | (df['Wind Direction'] > 360), np.nan)
df['Humidity'] = df['Humidity'].mask((df['Humidity'] < 0) | (df['Humidity'] > 100), np.nan)

In [62]:
df[df.isna().any(axis=1)]

,DateTime,Year,Month,Date,Time,Minute,Temperature,Wind Speed,Wind Direction,Rainfall
36,2008-01-02 12:00:00,2008.0,1.0,2.0,12.0,1.0,NaN,NaN,NaN,NaN
384,2008-01-17 00:00:00,2008.0,1.0,17.0,0.0,1.0,6.2,NaN,NaN,NaN
665,2008-01-28 17:00:00,2008.0,1.0,28.0,17.0,1.0,14.2,53.0,190.0,NaN
950,2008-02-09 14:00:00,2008.0,2.0,9.0,14.0,1.0,NaN,NaN,NaN,NaN
951,2008-02-09 15:00:00,2008.0,2.0,9.0,15.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
135115,2023-05-31 19:00:00,2023.0,5.0,31.0,19.0,1.0,25.6,26.0,NaN,0.0
135116,2023-05-31 20:00:00,2023.0,5.0,31.0,20.0,1.0,25.7,24.0,NaN,0.0
135117,2023-05-31 21:00:00,2023.0,5.0,31.0,21.0,1.0,25.6,14.0,NaN,0.0
135118,2023-05-31 22:00:00,2023.0,5.0,31.0,22.0,1.0,25.4,1.0,NaN,0.0


In [63]:
df = df.fillna(method="ffill")

Prepare the data for LSTM

Remove rows with missing values

In [64]:
df = df.dropna().reset_index()

In [65]:
df

,index,DateTime,Year,Month,Date,Time,Minute,Temperature,Wind Speed,Wind Direction,Rainfall
0,0,2008-01-01 00:00:00,2008.0,1.0,1.0,0.0,1.0,3.6,107.0,49.0,0.0
1,1,2008-01-01 01:00:00,2008.0,1.0,1.0,1.0,1.0,3.0,103.0,42.0,0.0
2,2,2008-01-01 02:00:00,2008.0,1.0,1.0,2.0,1.0,2.3,101.0,39.0,0.0
3,3,2008-01-01 03:00:00,2008.0,1.0,1.0,3.0,1.0,2.4,92.0,47.0,0.0
4,4,2008-01-01 04:00:00,2008.0,1.0,1.0,4.0,1.0,2.0,102.0,47.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
135834,135834,2023-06-30 18:00:00,2023.0,6.0,30.0,18.0,1.0,23.0,27.0,194.0,0.0
135835,135835,2023-06-30 19:00:00,2023.0,6.0,30.0,19.0,1.0,22.8,41.0,191.0,0.0
135836,135836,2023-06-30 20:00:00,2023.0,6.0,30.0,20.0,1.0,22.5,17.0,150.0,0.0
135837,135837,2023-06-30 21:00:00,2023.0,6.0,30.0,21.0,1.0,21.8,30.0,166.0,0.0


Check if any hours are missing

In [66]:
# Get the minimum and maximum DateTime values in the DataFrame
min_time = df['DateTime'].min()
max_time = df['DateTime'].max()

# Create a complete sequence of hours using pandas date_range
complete_hours = pd.date_range(start=min_time, end=max_time, freq='H')

# Extract the unique hours from the DataFrame
existing_hours = pd.to_datetime(df['DateTime']).unique()

# Find the missing hours by comparing the complete sequence with the existing hours
missing_hours = [hour for hour in complete_hours if hour not in existing_hours]

# Print the missing hours
print(missing_hours)

[]


In [67]:
time_steps = 72  # Number of time steps for the LSTM model
scaler = MinMaxScaler(feature_range=(0, 1))  # Scale the data to [0, 1]

In [68]:
features_keys = ['Temperature', 'Wind Speed',
               'Rainfall', 'Humidity']

Scale the temperature, wind speed, rainfall, and wind direction values

In [69]:
features = df[features_keys].copy()
scaled_features = scaler.fit_transform(features)

In [70]:
scaled_features

array([[0.27350427, 0.21730382, 0.00110988],
       [0.25641026, 0.20925553, 0.00110988],
       [0.23646724, 0.20523139, 0.00110988],
       ...,
       [0.81196581, 0.0362173 , 0.00110988],
       [0.79202279, 0.06237425, 0.00110988],
       [0.79202279, 0.07243461, 0.00110988]])

Create sequences of input data and corresponding target values

In [71]:
data = []
target = []
for i in range(1, len(scaled_features) - time_steps):
    # exclude the target
    data.append(scaled_features[i:i+time_steps])
    # Only the temperature is the target
    target.append(scaled_features[i+time_steps])

In [72]:
data = np.array(data)
target = np.array(target)


Split the dataset into training and validation sets

In [73]:
split = 0.1
exclude_index = int(len(data) * (1-split))
train_data, train_target = data[:exclude_index], target[:exclude_index]
val_data, val_target = data[exclude_index:], target[exclude_index:]


Build the LSTM model architecture

In [74]:
import tensorflow as tf
"""
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(256, return_sequences=True,
                         input_shape=(time_steps, 4)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(256, return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(256, return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(256),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])
"""
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(256, return_sequences=True,  input_shape=(time_steps, len(features_keys))),
    # tf.keras.layers.LSTM(128, return_sequences=True),
    # tf.keras.layers.LSTM(64, return_sequences=True),
    # tf.keras.layers.LSTM(64, return_sequences=True),
    # tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(64),
    # tf.keras.layers.Dense(64),
    # tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(len(features_keys))
])


Compile the model

In [75]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_absolute_error', optimizer=optimizer)

Define early stopping and learning rate scheduler

In [76]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

Train the LSTM model

In [ ]:
batch_size = 4096
epochs = 150
history = model.fit(train_data, train_target, batch_size=batch_size, epochs=epochs, validation_data=(val_data, val_target),
                    callbacks=[early_stopping, lr_scheduler])

Epoch 1/100
30/30 [==============================] - 47s 800ms/step - loss: 0.1609 - val_loss: 0.0781 - lr: 0.0100
Epoch 2/100
30/30 [==============================] - 18s 593ms/step - loss: 0.0770 - val_loss: 0.0688 - lr: 0.0100
Epoch 3/100
30/30 [==============================] - 17s 582ms/step - loss: 0.0699 - val_loss: 0.0697 - lr: 0.0100
Epoch 4/100
30/30 [==============================] - 17s 574ms/step - loss: 0.0693 - val_loss: 0.0708 - lr: 0.0100
Epoch 5/100
30/30 [==============================] - 17s 575ms/step - loss: 0.0627 - val_loss: 0.0583 - lr: 0.0100
Epoch 6/100
30/30 [==============================] - 18s 586ms/step - loss: 0.0519 - val_loss: 0.0508 - lr: 0.0100
Epoch 7/100
30/30 [==============================] - 17s 584ms/step - loss: 0.0495 - val_loss: 0.0530 - lr: 0.0100
Epoch 8/100
30/30 [==============================] - 17s 584ms/step - loss: 0.0503 - val_loss: 0.0513 - lr: 0.0100
Epoch 9/100
30/30 [==============================] - 17s 579ms/step - loss: 0.04

Make predictions using the trained LSTM model

In [ ]:
predictions = model.predict(val_data)

Rescale the predictions back to the original range

In [ ]:
scaled_predictions = pd.DataFrame(predictions, columns=features_keys)
predictions = pd.DataFrame(scaler.inverse_transform(
    predictions), columns=features_keys)
predicted_temperature = predictions["Temperature"]


In [ ]:
val_target = pd.DataFrame(val_target, columns=features_keys)
val_target_rescaled = pd.DataFrame(scaler.inverse_transform(val_target), columns=features_keys)
train_target = pd.DataFrame(train_target, columns=features_keys)
train_target_rescaled = pd.DataFrame(scaler.inverse_transform(train_target), columns=features_keys)

Calculate MSE and MAE

In [ ]:
mse = mean_squared_error(val_target_rescaled["Temperature"], predictions["Temperature"])
mae = mean_absolute_error(val_target_rescaled["Temperature"], predictions["Temperature"])

In [ ]:
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)

Visualize the actual vs. predicted temperatures for the validation set

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(len(val_target)), val_target_rescaled["Temperature"], label='Actual')
plt.plot(range(len(val_target)), predicted_temperature, label='Predicted')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.title('Actual vs. Predicted Temperatures (Validation Set)')
plt.legend()
plt.show()

Visualize the actual vs. predicted temperatures for the training set

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(len(train_target)), train_target["Temperature"], label='Actual')
plt.plot(range(len(train_target)), pd.DataFrame(model.predict(
    train_data), columns=features_keys)["Temperature"], label='Predicted')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.title('Actual vs. Predicted Temperatures (Training Set)')
plt.legend()
plt.show()

Save the trained model

In [ ]:
model.save("lstm.keras")

In [ ]:
scaled_features
scaled_features.copy()[-time_steps:].shape

In [ ]:
from datetime import timedelta
model = tf.keras.models.load_model("lstm.keras")

data = scaled_features.copy()[-time_steps:]
predicted = []

for i in range(1, 31):
    scaled_prediction = model.predict(data.reshape(1, time_steps, len(features_keys)))
    prediction = scaler.inverse_transform(scaled_prediction)[0]
    print(f'{df["DateTime"].max() + timedelta(hours=i)}:', list(map(lambda x: round(x, 6), prediction)))
    # print(prediction[0], data, np.append(data, prediction[0]))
    data = np.vstack((data, scaled_prediction))
    data = np.delete(data, 0, axis=0)
    predicted.append(prediction)

In [ ]:
results = pd.DataFrame(predicted, columns=features_keys, index=[df["DateTime"].max() + timedelta(hours=i) for i in range(1, 31)])
results.to_csv("predict.csv")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(30), np.array(predicted)[:, 0], label='Predicted')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.title('Predicted Temperatures')
plt.legend()
plt.show()